# Pocessing GWO hourly meteorological data
**Author: Jun Sasaki  Coded on 2022-02-13  Updated on 2024-12-20.**<br>
Extract and plot GWO (Ground Weather Observation) hourly data.

In [ ]:
from metdata import gwo
from datetime import datetime
from dateutil.parser import parse
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from matplotlib.dates import date2num, YearLocator, MonthLocator, DayLocator, DateFormatter
import pandas as pd
import numpy as np
from pandas.plotting import register_matplotlib_converters

# Set GWO/Hourly/ directory path containing GWO data
dirpath = "/mnt/d/dat/met/JMA_DataBase/GWO/Hourly/"
# %matplotlib inline

In [ ]:
datetime_ini = "2020-01-01 00:00:00"
datetime_end = "2021-01-01 00:00:00"
#datetime_ini = "2010-9-2 00:00:00"
#datetime_end = "2010-9-3 00:00:00"
stn = "Tokyo"
met = gwo.Hourly(datetime_ini=datetime_ini, datetime_end=datetime_end,
                 stn=stn, dirpath=dirpath)

## Gets pandas DataFrame
pandas DataFrame can be obtained by invoking `.df` method

In [ ]:
print(met.df.columns)
met.df.head()

# Plot using [Matplotlib](https://matplotlib.org/stable/index.html)
Extract a 1D scalar or vector variable from DataFrame.

In [ ]:
data = gwo.Data1D(df=met.df, col_1='kion')

## Example of scalar 1-D time series plot
- Rolling mean is applied to `Plot1D()` by setting its arguments of `window` in odd integer number and `center` (default is `True`).

In [ ]:
### xlim = (parse("2014-01-15"), parse("2014-02-16"))  ### ex. for datetime
ylabel='Temperature (degC)'
xlim = None
#xlim = (parse("1990-09-02"), parse("1992-09-03"))
dx = 7
ylim = None
dy = 2

## Set window=1 when no plot.
window=1
#try:
plot_config = gwo.Data1D_PlotConfig(xlim=xlim, ylim=ylim, 
                                    x_minor_locator=DayLocator(interval=dx),
                                    y_minor_locator = MultipleLocator(dy),
                                    format_xdata = DateFormatter('%Y-%m-%d'),
                                    ylabel = ylabel)
gwo.Plot1D(plot_config, data, window=window,
           center=True).save_plot('data.png', dpi=600)

## Example of time series wind vector plot with its speed

In [ ]:
wind = gwo.Data1D(met.df, 'u', 'v')
print(wind.v[0:10])

In [ ]:
#xlim = (parse("2013-12-25 00:00:00"),parse("2014-01-10 00:00:00"))
vlabel = 'Wind speed (m/s)'
ylabel = 'Wind vector (m/s)'
png_vector = "gwo_hourly_wind.png"
xlim = None
#xlim = (parse("1990-09-02"), parse("1990-09-03"))
dx = 7  # x_minor_locator interval
#ylim = None
ylim = (-15, 15)
dy = 1  # y_minor_locator interval
window=25 # Rolling mean window in odd integer; center: rolling mean at center
magnitude = True  # True: Plot magnitudes, False: No magnitudes

plot_config = gwo.Data1D_PlotConfig(xlim = xlim, ylim = ylim,
              x_minor_locator = DayLocator(interval=dx),
              y_minor_locator = MultipleLocator(dy),
              format_xdata = DateFormatter('%Y-%m-%d'),
              ylabel=ylabel, vlabel=vlabel, vlabel_loc = 'lower center')
gwo.Plot1D(plot_config, wind, window=window, 
           center=True).save_vector_plot(png_vector, 
                                         magnitude = magnitude, dpi=600)

# Plot using [hvPlot](https://hvplot.holoviz.org/)
This is for interactive plotting but not suitable for saving graphics into files.

In [ ]:
import hvplot.pandas

In [ ]:
data.df[['kion', 'sped']].hvplot()

In [ ]:
def hook(plot, element):
    plot.handles['xaxis'].axis_label_text_font_style = 'normal'
    plot.handles['yaxis'].axis_label_text_font_style = 'normal'

### How to specify options for hvPlot (オプション指定方法)
- Many of the Holoviews options can be specified with the hvPlot argument.
- `hooks` defines the function `hook(plot, element)`, which is specified in hvPlot as `.opts(hooks=[hook])`. Unconfirmed, but it seems that all options can be specified, including Bokeh options that are not defined as arguments in hvPlot or Holoviews.
- 基本的にはhvplotの引数でHoloviewsのオプションの多くが指定できる
- `hooks`は関数`hook(plot, element)`を定義し，hvPlotで`.opts(hooks=[hook])`として指定する．未確認だが，hvPlotやHoloviewsの引数としては定義されていないBokehのオプションを含め，すべてのオプションが指定できそう

In [ ]:
'''
## This cell invokes error.
xlim = (parse("2019-12-01"), parse("2019-12-05"))
data.df['kion'].hvplot(xlim=xlim)
'''

In [ ]:
data.df['kion']['2020-12-01':'2020-12-05'].hvplot()

In [ ]:
data.df['kion']['2020-12-01':'2020-12-05'].hvplot(
                       #xlim=(datetime(2019,12,1), datetime(2019,12,20)), 
                       #xticks=10, ylabel='Temperature (degC)',
                       #ylim=(-4,35), yticks=10, width=600, height=200,
                       line_color='red', line_width=0.5,
                       fontsize={'xticks':12,'yticks':12 ,'ylabel':14},
                       title='').opts(hooks=[hook])

# Check missing rows in DataFrame.

In [ ]:
datetime_ini = "2010-9-2 00:00:00"
datetime_end = "2010-9-2 23:00:00"
stn = "Chiba"
dirpath = "/mnt/d/dat/met/JMA_DataBase/GWO/Hourly/"

met_check = gwo.Check(datetime_ini=datetime_ini, datetime_end=datetime_end,
                      stn=stn, dirpath=dirpath)
## Create a complete pandas DatetieIndex
datetime_index = pd.date_range(datetime_ini, datetime_end, freq='h')

In [ ]:
met_check.df.index

In [ ]:
datetime_index.values[0] in met_check.df.index.values

### Create a mask for extracting missing rows
- [`np.isis(https://numpy.org/doc/stable/reference/generated/numpy.isin.html)`]()
- [`np.logical_not()`](https://numpy.org/doc/stable/reference/generated/numpy.logical_not.html).

In [ ]:
mask = np.logical_not(np.isin(datetime_index, met_check.df.index))
datetime_index[mask]

In [ ]:
mask

## Exporting GOTM input data

In [ ]:
light_file = "tokyo_light_2020.dat"
precip_file = "tokyo_precip_2020.dat"
meteo_file = "tokyo_meteo_2020.dat"
gwo.export_gotm(light_file, met.df, 'slht', fmt='10.5f')
gwo.export_gotm(precip_file, met.df, 'kous')
gwo.export_gotm_meteo(meteo_file, met.df)